In [1]:
import pygame
import math

# Инициализация PyGame
pygame.init()

# Размеры окна и сетки
GRID_SIZE = 10
CELL_SIZE = 50
WINDOW_SIZE = GRID_SIZE * CELL_SIZE

# Цвета
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Настройка окна
screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
pygame.display.set_caption("Tank Game")

# Класс для танка
class Tank:
    def __init__(self, x, y):
        self.x = x  # Координата x (в клетках)
        self.y = y  # Координата y (в клетках)
        self.angle = 0  # Угол поворота (в градусах)
        self.speed = 1  # Скорость движения (в клетках за шаг)

    def move_forward(self):
        new_x = self.x + math.cos(math.radians(self.angle)) * self.speed
        new_y = self.y - math.sin(math.radians(self.angle)) * self.speed
        if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE:
            self.x = new_x
            self.y = new_y

    def move_backward(self):
        new_x = self.x - math.cos(math.radians(self.angle)) * self.speed
        new_y = self.y + math.sin(math.radians(self.angle)) * self.speed
        if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE:
            self.x = new_x
            self.y = new_y

    def rotate_left(self):
        self.angle += 15  # Поворот на 15 градусов влево

    def rotate_right(self):
        self.angle -= 15  # Поворот на 15 градусов вправо

    def draw(self):
        # Рисуем танк как круг с "дулом"
        center_x = self.x * CELL_SIZE + CELL_SIZE // 2
        center_y = self.y * CELL_SIZE + CELL_SIZE // 2
        pygame.draw.circle(screen, BLUE, (int(center_x), int(center_y)), CELL_SIZE // 3)
        # Дуло танка
        end_x = center_x + math.cos(math.radians(self.angle)) * (CELL_SIZE // 3)
        end_y = center_y - math.sin(math.radians(self.angle)) * (CELL_SIZE // 3)
        pygame.draw.line(screen, BLACK, (center_x, center_y), (end_x, end_y), 3)

# Класс для цели
class Target:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self):
        center_x = self.x * CELL_SIZE + CELL_SIZE // 2
        center_y = self.y * CELL_SIZE + CELL_SIZE // 2
        pygame.draw.rect(screen, RED, (center_x - 10, center_y - 10, 20, 20))

# Инициализация объектов
tank = Tank(1, 8)  # Танк в позиции (1, 8), как на рисунке
target = Target(8, 1)  # Цель в позиции (8, 1), как на рисунке

# Препятствия (просто список координат стен)
obstacles = [(2, 2), (2, 3), (3, 2), (4, 4), (5, 5)]

# Основной игровой цикл
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_w:
                tank.move_forward()
            elif event.key == pygame.K_s:
                tank.move_backward()
            elif event.key == pygame.K_a:
                tank.rotate_left()
            elif event.key == pygame.K_d:
                tank.rotate_right()

    # Очистка экрана
    screen.fill(WHITE)

    # Отрисовка сетки
    for i in range(GRID_SIZE):
        pygame.draw.line(screen, BLACK, (i * CELL_SIZE, 0), (i * CELL_SIZE, WINDOW_SIZE))
        pygame.draw.line(screen, BLACK, (0, i * CELL_SIZE), (WINDOW_SIZE, i * CELL_SIZE))

    # Отрисовка препятствий
    for obs in obstacles:
        pygame.draw.rect(screen, BLACK, (obs[0] * CELL_SIZE, obs[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Отрисовка танка и цели
    tank.draw()
    target.draw()

    # Обновление экрана
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

ModuleNotFoundError: No module named 'pygame'

In [ ]:
def get_state(tank, target, obstacles):
    # Расстояние до цели
    distance_to_target = math.sqrt((tank.x - target.x) ** 2 + (tank.y - target.y) ** 2)
    
    # Угол до цели
    angle_to_target = math.degrees(math.atan2(target.y - tank.y, target.x - tank.x))
    relative_angle = (angle_to_target - tank.angle) % 360
    
    # Расстояние до ближайших препятствий в 4 направлениях
    distances = {"up": GRID_SIZE, "down": GRID_SIZE, "left": GRID_SIZE, "right": GRID_SIZE}
    for obs in obstacles:
        if obs[0] == int(tank.x):  # Проверка по Y
            if obs[1] > tank.y:  # Вниз
                distances["down"] = min(distances["down"], obs[1] - tank.y)
            else:  # Вверх
                distances["up"] = min(distances["up"], tank.y - obs[1])
        if obs[1] == int(tank.y):  # Проверка по X
            if obs[0] > tank.x:  # Вправо
                distances["right"] = min(distances["right"], obs[0] - tank.x)
            else:  # Влево
                distances["left"] = min(distances["left"], tank.x - obs[0])
    
    return {
        "tank_pos": (tank.x, tank.y),
        "tank_angle": tank.angle,
        "distance_to_target": distance_to_target,
        "angle_to_target": relative_angle,
        "obstacle_distances": distances
    }

In [ ]:
def calculate_reward(tank, target, obstacles, prev_distance, action):
    reward = 0
    new_distance = math.sqrt((tank.x - target.x) ** 2 + (tank.y - target.y) ** 2)
    
    # Штраф за каждый шаг
    reward -= 1
    
    # Приближение/удаление от цели
    if new_distance < prev_distance:
        reward += 10
    else:
        reward -= 5
    
    # Столкновение с препятствием
    for obs in obstacles:
        if abs(tank.x - obs[0]) < 0.5 and abs(tank.y - obs[1]) < 0.5:
            reward -= 10
            break
    
    # Попадание в цель (предположим, стрельба успешна, если танк рядом с целью)
    if action == "shoot" and new_distance < 1:
        reward += 100
    
    return reward, new_distance

In [ ]:
prev_distance = math.sqrt((tank.x - target.x) ** 2 + (tank.y - target.y) ** 2)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            action = None
            if event.key == pygame.K_w:
                tank.move_forward()
                action = "move_forward"
            elif event.key == pygame.K_s:
                tank.move_backward()
                action = "move_backward"
            elif event.key == pygame.K_a:
                tank.rotate_left()
                action = "rotate_left"
            elif event.key == pygame.K_d:
                tank.rotate_right()
                action = "rotate_right"
            elif event.key == pygame.K_SPACE:
                action = "shoot"

            if action:
                state = get_state(tank, target, obstacles)
                reward, prev_distance = calculate_reward(tank, target, obstacles, prev_distance, action)
                print(f"State: {state}, Reward: {reward}")

    # Отрисовка (как ранее)
    screen.fill(WHITE)
    for i in range(GRID_SIZE):
        pygame.draw.line(screen, BLACK, (i * CELL_SIZE, 0), (i * CELL_SIZE, WINDOW_SIZE))
        pygame.draw.line(screen, BLACK, (0, i * CELL_SIZE), (WINDOW_SIZE, i * CELL_SIZE))
    for obs in obstacles:
        pygame.draw.rect(screen, BLACK, (obs[0] * CELL_SIZE, obs[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))
    tank.draw()
    target.draw()
    pygame.display.flip()
    clock.tick(30)

pygame.quit()